In [49]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
np.set_printoptions(suppress = True)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import roc_curve
from scipy.spatial.distance import cityblock, euclidean

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/keystroke-dynamics-benchmark-data-set/DSL-StrongPasswordData.csv


In [50]:
data = pd.read_csv("/kaggle/input/keystroke-dynamics-benchmark-data-set/DSL-StrongPasswordData.csv")
data.head()

,subject,sessionIndex,rep,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,...,H.a,DD.a.n,UD.a.n,H.n,DD.n.l,UD.n.l,H.l,DD.l.Return,UD.l.Return,H.Return
0,s002,1,1,0.1491,0.3979,0.2488,0.1069,0.1674,0.0605,0.1169,...,0.1349,0.1484,0.0135,0.0932,0.3515,0.2583,0.1338,0.3509,0.2171,0.0742
1,s002,1,2,0.1111,0.3451,0.2340,0.0694,0.1283,0.0589,0.0908,...,0.1412,0.2558,0.1146,0.1146,0.2642,0.1496,0.0839,0.2756,0.1917,0.0747
2,s002,1,3,0.1328,0.2072,0.0744,0.0731,0.1291,0.0560,0.0821,...,0.1621,0.2332,0.0711,0.1172,0.2705,0.1533,0.1085,0.2847,0.1762,0.0945
3,s002,1,4,0.1291,0.2515,0.1224,0.1059,0.2495,0.1436,0.1040,...,0.1457,0.1629,0.0172,0.0866,0.2341,0.1475,0.0845,0.3232,0.2387,0.0813
4,s002,1,5,0.1249,0.2317,0.1068,0.0895,0.1676,0.0781,0.0903,...,0.1312,0.1582,0.0270,0.0884,0.2517,0.1633,0.0903,0.2517,0.1614,0.0818


In [51]:
def evaluate(subjects):
    eers = []

    for subject in subjects:
        genuine_user_data = data.loc[data.subject == subject, \
                                     "H.period":"H.Return"]
        imposter_data = data.loc[data.subject != subject, :]

        train = genuine_user_data[:200]
        test_genuine = genuine_user_data[200:]
        test_imposter = imposter_data.groupby("subject"). \
                             head(5).loc[:, "H.period":"H.Return"]

        mean_vector = ManhattanFiltered(train)
        user_scores,imposter_scores = testing(test_genuine,test_imposter,mean_vector)
        eers.append(evaluateEER(user_scores, \
                                 imposter_scores))
    return np.mean(eers)

In [52]:
def testing(test_genuine,test_imposter,mean_vector):
    user_scores = []
    imposter_scores = []
    for i in range(test_genuine.shape[0]):
        cur_score = cityblock(test_genuine.iloc[i].values, \
                               mean_vector)
        user_scores.append(cur_score)

    for i in range(test_imposter.shape[0]):
        cur_score = cityblock(test_imposter.iloc[i].values, \
                               mean_vector)
        imposter_scores.append(cur_score)
    return user_scores,imposter_scores


In [53]:
 def Manhattan(train):
    mean_vector = train.mean().values
    return mean_vector

In [54]:
def evaluateEER(user_scores, imposter_scores):
    labels = [0]*len(user_scores) + [1]*len(imposter_scores)
    fpr, tpr, thresholds = roc_curve(labels, user_scores + imposter_scores)

    missrates = 1 - tpr
    farates = fpr
    dists = missrates - farates
    
    idx1 = np.argmin(dists[dists >= 0])
    idx2 = np.argmax(dists[dists < 0])
    x = [missrates[idx1], farates[idx1]]
    y = [missrates[idx2], farates[idx2]]
    a = ( x[0] - x[1] ) / ( y[1] - x[1] - y[0] + x[0] )
    eer = x[0] + a * ( y[0] - x[0] )
    return eer


In [37]:
subjects = data["subject"].unique()
print("EER for Manhattan Detector is" ,evaluate(subjects))

EER for Manhattan Detector is 0.15026997944690795
